In [160]:
sc

In [161]:
movies = sc.textFile("file:///home/modekus/Project/input/small2.txt")
#movies = sc.textFile("file:///home/modekus/Project/input/movies.txt")

In [162]:
movies.take(1)

[u'product/productId: B003AI2VGA']

In [163]:
def get_line_info(row):
    
    if not row:
        return (-1, None)
    
    (review_code, review_value) = row.split(': ', 1)

    if review_code == 'product/productId':
        return (1, review_value)
    if review_code == 'review/userId':
        return (2, review_value)
    if review_code == 'review/profileName':
        return (3, review_value)
    if review_code == 'review/helpfulness':
        return (4, review_value)
    if review_code == 'review/score':
        return (5, review_value)
    if review_code == 'review/time':
        return (6, review_value)
    if review_code == 'review/summary':
        return (7, review_value)
    if review_code == 'review/text':
        return (8, review_value)
    return (-1, None)

In [164]:
parsemovies = movies.map(lambda x : get_line_info(x))

In [165]:
parsemovies.take(10)

[(1, u'B003AI2VGA'),
 (2, u'A141HP4LYPWMSR'),
 (3, u'Brian E. Erland "Rainbow Sphinx"'),
 (4, u'7/7'),
 (5, u'3.0'),
 (6, u'1182729600'),
 (7, u'"There Is So Much Darkness Now ~ Come For The Miracle"'),
 (8,
  u'Synopsis: On the daily trek from Juarez, Mexico to El Paso, Texas an ever increasing number of female workers are found raped and murdered in the surrounding desert. Investigative reporter Karina Danes (Minnie Driver) arrives from Los Angeles to pursue the story and angers both the local police and the factory owners who employee the undocumented aliens with her pointed questions and relentless quest for the truth.<br /><br />Her story goes nationwide when a young girl named Mariela (Ana Claudia Talancon) survives a vicious attack and walks out of the desert crediting the Blessed Virgin for her rescue. Her story is further enhanced when the "Wounds of Christ" (stigmata) appear in her palms. She also claims to have received a message of hope for the Virgin Mary and soon a fanati

In [166]:

def floatconv(x):
    try:
        t=float(x[0])/float(x[1])
    except:
        t=1.0
    return t
ratingusefuldata=parsemovies.filter(lambda x: x[0] in [4]).map(lambda x: x[1].split('/')).filter(lambda x:len(x)==2).map(lambda x:floatconv(x) )

In [167]:
ratingusefuldata.take(10)

[1.0,
 1.0,
 0.8,
 1.0,
 1.0,
 1.0,
 0.2727272727272727,
 0.9846153846153847,
 1.0,
 1.0]

In [168]:
ratingdata=parsemovies.filter(lambda x: x[0] in [5]).map(lambda x: x[1])

In [169]:
ratingdata.take(10)

[u'3.0',
 u'3.0',
 u'5.0',
 u'3.0',
 u'3.0',
 u'2.0',
 u'1.0',
 u'5.0',
 u'5.0',
 u'5.0']

In [170]:
data = ratingdata.zip(ratingusefuldata).filter(lambda x:len(x)==2)

In [171]:
data.take(10)

[(u'3.0', 1.0),
 (u'3.0', 1.0),
 (u'5.0', 0.8),
 (u'3.0', 1.0),
 (u'3.0', 1.0),
 (u'2.0', 1.0),
 (u'1.0', 0.2727272727272727),
 (u'5.0', 0.9846153846153847),
 (u'5.0', 1.0),
 (u'5.0', 1.0)]

In [172]:
resultDF = sqlContext.createDataFrame(data,['rating','usefulness'])

In [173]:
resultDF.registerTempTable('ratings')

In [176]:
l=sqlContext.sql("select rating,sum(usefulness)/count(*) as  useful from ratings group by rating limit 10")

In [177]:
l.show(10)

+------+-------------------+
|rating|             useful|
+------+-------------------+
|   1.0|0.46896949068683474|
|   5.0|  0.791073473366171|
|   4.0| 0.7666482915984856|
|   2.0|  0.530552293186399|
|   3.0| 0.6293620145782315|
+------+-------------------+

